# AI Music Compositor using LangGraph

## Overview
This tutorial demonstrates how to build an AI-powered music composition system using LangGraph, a framework for creating workflows with language models. The system generates musical compositions based on user input, leveraging various components to create melody, harmony, rhythm, and style adaptations.

## Motivation
Creating music programmatically is a fascinating intersection of artificial intelligence and artistic expression. This project aims to explore how language models and graph-based workflows can be used to generate coherent musical pieces, providing a unique approach to AI-assisted music composition.

## Key Components
1. State Management: Utilizes a `MusicState` class to manage the workflow's state.
2. Language Model: Employs ChatOpenAI (GPT-4) for generating musical components.
3. Musical Functions:
   - Melody Generator
   - Harmony Creator
   - Rhythm Analyzer
   - Style Adapter
4. MIDI Conversion: Transforms the composition into a playable MIDI file.
5. LangGraph Workflow: Orchestrates the composition process using a state graph.

## Method
1. The workflow begins by generating a melody based on user input.
2. It then creates harmony to complement the melody.
3. A rhythm is analyzed and suggested for the melody and harmony.
4. The composition is adapted to the specified musical style.
5. The final composition is converted to MIDI format.

The entire process is orchestrated using LangGraph, which manages the flow of information between different components and ensures that each step builds upon the previous ones.

## Conclusion
This AI Music Compositor demonstrates the potential of combining language models with structured workflows to create musical compositions. By breaking down the composition process into discrete steps and leveraging the power of AI, we can generate unique musical pieces based on simple user inputs. This approach opens up new possibilities for AI-assisted creativity in music production and composition.

## Imports

Import all necessary modules and libraries for the AI Music Collaborator.

In [3]:
# Import required libraries
from typing import Dict, TypedDict
from langgraph.graph import StateGraph, END
from langchain.prompts import ChatPromptTemplate
import music21
from music21 import midi
import tempfile
import os
import random
from openai import AzureOpenAI
from langchain_openai import AzureChatOpenAI
from langgraph.checkpoint.memory import MemorySaver


## State Definition

Define the MusicState class to hold the workflow's state.

In [4]:
class MusicState(TypedDict):
    """Define the structure of the state for the music generation workflow."""
    musician_input: str  # User's input describing the desired music
    melody: str          # Generated melody
    harmony: str         # Generated harmony
    rhythm: str          # Generated rhythm
    style: str           # Desired musical style
    composition: str     # Complete musical composition
    midi_file: str       # Path to the generated MIDI file

## LLM Initialization

Initialize the Language Model (LLM) for generating musical components.

In [5]:
api_key = "YOUR_API_KEY"
endpoint = "YOUR_API_ENDPOINT"

llm = AzureChatOpenAI(
    azure_deployment="gpt-4o",  # or your deployment
    api_version="2024-08-01-preview",  # or your api version
    temperature=0,
    api_key = api_key,
    azure_endpoint=endpoint,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)


## Component Functions

Define the component functions for melody generation, harmony creation, rhythm analysis, style adaptation, and MIDI conversion.

In [16]:
def melody_generator(state: MusicState) -> Dict:
    """Generate a melody based on the user's input."""
    prompt = ChatPromptTemplate.from_template(
        "Generate a melody based on this input: {input}. Represent it as a string of notes in music21 format and only provide executable code nothing else."
    )
    chain = prompt | llm
    melody = chain.invoke({"input": state["musician_input"]})
    return {"melody": melody.content}

def harmony_creator(state: MusicState) -> Dict:
    """Create harmony for the generated melody."""
    prompt = ChatPromptTemplate.from_template(
        "Create harmony for this melody: {melody}. Represent it as a string of chords in music21 format and only provide executable code nothing else."
    )
    chain = prompt | llm
    harmony = chain.invoke({"melody": state["melody"]})
    return {"harmony": harmony.content}

def rhythm_analyzer(state: MusicState) -> Dict:
    """Analyze and suggest a rhythm for the melody and harmony."""
    prompt = ChatPromptTemplate.from_template(
        "Analyze and suggest a rhythm for this melody and harmony: {melody}, {harmony}. Represent it as a string of durations in music21 format and only provide executable code nothing else."
    )
    chain = prompt | llm
    rhythm = chain.invoke({"melody": state["melody"], "harmony": state["harmony"]})
    return {"rhythm": rhythm.content}

def style_adapter(state: MusicState) -> Dict:
    """Adapt the composition to the specified musical style."""
    prompt = ChatPromptTemplate.from_template(
        "Adapt this composition to the {style} style: Melody: {melody}, Harmony: {harmony}, Rhythm: {rhythm}. Provide the result in midi format and define a function to save the score and return the MIDI file path. Save the MIDI file and return the path. only provide executable code nothing else also dont include the backticks as well as I want to directly run it without any changes. "
    )
    chain = prompt | llm
    adapted = chain.invoke({
        "style": state["style"],
        "melody": state["melody"],
        "harmony": state["harmony"],
        "rhythm": state["rhythm"]
    })
    return {"composition": adapted.content}

def midi_converter(state: MusicState) -> Dict:
    """Convert the composition to MIDI format and save it as a file."""
    # Retrieve the generated composition code from the state
    composition_code = state["composition"]
    
    # Initialize a dictionary to hold the execution context
    exec_context = {}

    try:
        # Execute the composition code in a safe context
        
        exec(composition_code, exec_context)
        midi_file_path = exec_context.get('midi_file_path')
        return {"midi_file": midi_file_path}

    except Exception as e:
        return {"midi_file": f"Error executing composition code: {str(e)}"}


## Graph Construction

Construct the LangGraph workflow for the AI Music Collaborator.

In [17]:
# Initialize the StateGraph
workflow = StateGraph(MusicState)

# Add nodes to the graph
workflow.add_node("melody_generator", melody_generator)
workflow.add_node("harmony_creator", harmony_creator)
workflow.add_node("rhythm_analyzer", rhythm_analyzer)
workflow.add_node("style_adapter", style_adapter)
workflow.add_node("midi_converter", midi_converter)

# Set the entry point of the graph
workflow.set_entry_point("melody_generator")

# Add edges to connect the nodes
workflow.add_edge("melody_generator", "harmony_creator")
workflow.add_edge("harmony_creator", "rhythm_analyzer")
workflow.add_edge("rhythm_analyzer", "style_adapter")
workflow.add_edge("style_adapter", "midi_converter")
workflow.add_edge("midi_converter", END)

# Compile the graph
memory = MemorySaver()

app = workflow.compile(checkpointer=memory)

## Run the Workflow

Execute the AI Music Collaborator workflow to generate a musical composition.

In [18]:
# Define input parameters
inputs = {
    "musician_input": "Create a happy piano piece in C major",
    "style": "Romantic era"
}
config = {"configurable": {"thread_id": "1"}}

# Invoke the workflow
result = app.invoke(inputs, config)

print(f"MIDI file saved at: {result['midi_file']}")

MIDI file saved at: romantic_melody.mid
